[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/arturoTellez/tareas_AP/blob/master/RedesDensas/Respuestas.ipynb)

# Curso de aprendizaje profundo
PCIC, UNAM  
Tarea 1: Redes densas*

In [1]:
# !pip install plotly
# !pip install torch torchvision torchaudio
# !pip install tensorflow
# !pip install scipy
# !pip install ipykernel --update-deps --force-reinstall
# !python -m pip install 'traitlets==4.3.3' --force-reinstall

In [2]:
# Se importan los paquetes necesarios
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import plotly.express as px
from scipy import optimize
style.use("ggplot")
%matplotlib inline


import math
# sistema de archivos
import os
# números aleatorios
import random

%matplotlib
# gráficas
import matplotlib.pyplot as plt
# arreglos multidimensionales
import numpy as np
# csv
import pandas as pd

# redes neuronales
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader, TensorDataset
%matplotlib inline



Using matplotlib backend: Qt5Agg


### 1. Red de unidades de umbral lineal
Programa y evalúa una red de neuronas con funciones de activación escalón unitario que aproxime la operación XNOR (⊙) dada por  
|x1|x2|y |
|--|--|--|
|0 |0 |1 |
|0 |1 |0 |
|1 |0 |0 |
|1 |1 |1 |

In [3]:
def escalon(z):
    return int(z>=0)

In [4]:
X = np.array([[0., 0.], [0., 1.], [1., 0.], [1., 1.]])
y = np.array([1., 0., 0., 1.])

In [5]:
def multicapa(x, W1, b1, W2, b2):
  escv = np.vectorize(escalon)
  a = escv(np.dot(W1.T, x) + b1)
  return escv(np.dot(W2.T, a) + b2)

def buscar_pesos(var,  x, y):
    """
    Función para encontrar los parámetros
    """
    w11, w12, w13, w14, b11, b12, w21, w22, b2 = var
    W1 = np.array([[w11, w12], [w13, w14]])
    W2 = np.array([[w21], [w22]])
    b1 = np.array([b11, b12])
    b2 = np.array([b2])

    return  sum([multicapa(x[i,:], W1, b1, W2, b2)[0] != y[i] for i in range(len(y))])

buscar_pesos((20,30,10,40,50,80,20, 50,60 ), X, y,)

    

2

In [6]:
optimo = 4
while optimo != 0:
  res = optimize.minimize(
      buscar_pesos,
      method='Nelder-Mead',
      # bounds = ((-100,100), (-100,100),(-100,100),(-100,100),(-100,100), (-100,100),(-100,100), (-100,100), (-100,100)),
      tol = 0.00000001,
      x0=[np.random.random(9) * 20 - 10] ,
      args=(X, y),
      options = {"maxiter":1000000000}
  )
  if res.success:
    optimo = res.fun
print(res)

 final_simplex: (array([[ 9.2042442 , -6.79105682, -5.3577791 ,  5.35077911,  1.45037076,
         1.82074128,  5.93761881,  4.02581137, -7.81421877],
       [ 9.20424421, -6.79105682, -5.3577791 ,  5.35077911,  1.45037076,
         1.82074128,  5.93761881,  4.02581137, -7.81421877],
       [ 9.2042442 , -6.79105682, -5.3577791 ,  5.35077911,  1.45037076,
         1.82074128,  5.93761881,  4.02581137, -7.81421877],
       [ 9.2042442 , -6.79105682, -5.3577791 ,  5.35077911,  1.45037076,
         1.82074128,  5.93761881,  4.02581137, -7.81421877],
       [ 9.2042442 , -6.79105682, -5.3577791 ,  5.35077911,  1.45037076,
         1.82074128,  5.93761881,  4.02581137, -7.81421877],
       [ 9.2042442 , -6.79105682, -5.3577791 ,  5.35077911,  1.45037076,
         1.82074128,  5.93761881,  4.02581137, -7.81421877],
       [ 9.2042442 , -6.79105682, -5.3577791 ,  5.35077911,  1.45037076,
         1.82074128,  5.93761881,  4.02581137, -7.81421877],
       [ 9.2042442 , -6.79105682, -5.3577791 

In [7]:
buscar_pesos(res.x,X,y)


0

In [8]:
w11, w12, w13, w14, b11, b12, w21, w22, b2 = res.x
X = X
y_xnor = y.copy()
W1 = np.array([[w11, w12], [w13, w14]])
W2 = np.array([[w21], [w22]])
b1 = np.array([b11, b12])
b2 = np.array([b2])



print('W_1 = [{0}{1}], b_1 = {2}'.format(W1[0, :], W1[1, :], b1))
print('W_2 = [{0}{1}], b_2 = {2}'.format(W2[0], W2[1], b2))
print('-----------------------------')
print('x_1 \tx_2 \ty\ty_hat')
print('-----------------------------')
for i in range(X.shape[0]):
  y_hat = multicapa(X[i], W1, b1, W2, b2)
  print('{0}\t{1}\t{2}\t{3}'.format(X[i, 0], X[i, 1], y_xnor[i], y_hat[0]))

W_1 = [[ 9.2042442  -6.79105682][-5.3577791   5.35077911]], b_1 = [1.45037076 1.82074128]
W_2 = [[5.93761881][4.02581137]], b_2 = [-7.81421877]
-----------------------------
x_1 	x_2 	y	y_hat
-----------------------------
0.0	0.0	1.0	1
0.0	1.0	0.0	0
1.0	0.0	0.0	0
1.0	1.0	1.0	1


### 3. Regresión lineal con PyTorch
Entrena y evalúa un modelo de regresión lineal para el conjunto de datos de calificaciones que considere los dos atributos de entrada.  
Especificaciones:  
• Grafica los datos en 3D.  
• Realiza la definición de la arquitectura usando tanto la interfaz de alto como la de medio nivel.  
• Prueba distintos valores para los hiperparámetros.  
• Realiza una predicción con tu modelo entrenado para la calificación que tendría un alumno si sabemos que estudió durante 12 horas para el examen y en un examen previo obtuvo 3 de calificación.  
• Discute las modificaciones que realizaste y las distintas configuraciones para los hiperpa-
rámetros que probaste.

In [9]:
calif = pd.read_csv("https://raw.githubusercontent.com/gibranfp/CursoAprendizajeProfundo/2022-1/data/califs.csv")
display(calif.info())
print("=================================")
# display(calif.isnull().sum())
# sns.pairplot(calif)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   prev    50 non-null     float64
 1   horas   50 non-null     float64
 2   calif   50 non-null     float64
dtypes: float64(3)
memory usage: 1.3 KB


None

In [10]:
fig = px.scatter_3d(calif, x='prev', y='horas', z='calif')
fig.show()

In [11]:
def set_seed(seed=0):
    """Initializes pseudo-random number generators."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

In [12]:
# se obtiene el atributo de entrada y se agrega una dimensión
x_trn = np.array(calif.iloc[:,:2], dtype='float32')[..., np.newaxis]
# se obtiene la salda
y_trn = np.array(calif.iloc[:,-1], dtype='float32')[..., np.newaxis]
# graficamos
# plt.plot(x_trn[:,0], y_trn, '.', color='m', markersize=8)
# plt.xlabel('horas de estudio')
# plt.ylabel('calificación')
# plt.show();

In [17]:
x_trn = np.array(calif.iloc[:,:2], dtype="float32")
y_trn = np.array(calif.iloc[:,-1], dtype="float32")[..., np.newaxis]

x_trn = torch.tensor(x_trn)
y_trn = torch.tensor(y_trn)

print(x_trn.shape)
print(y_trn.shape)
ds = TensorDataset(x_trn, y_trn)
ds[0]
def build_dl(batch_size=16, shuffle=True):
    return DataLoader(ds, batch_size=batch_size, shuffle=True)

# creamos un DataLoader
dl = build_dl()

x, y = next(iter(dl))
print(f'x shape={x.shape} dtype={x.dtype}')
print(f'y shape={y.shape} dtype={y.dtype}')

# optimizador
def train(model, dl, epochs=5):

    opt = optim.SGD(model.parameters(), lr=1e-3)

    # historial de pérdida
    loss_hist = []

    # ciclo de entrenamiento
    for epoch in range(epochs):

        # historial
        loss_hist = []
        
        # entrenamiento de una época
        for x, y_true in dl:
            # hacemos inferencia para obtener los logits
            y_lgts = model(x)
            # calculamos de pérdida
            loss = F.mse_loss(y_lgts, y_true)
            # vaciamos los gradientes
            opt.zero_grad()
            # retropropagamos
            loss.backward()
            # actulizamos parámetros
            opt.step()

            # guardamos historial de pérdida
            loss_hist.append(loss.item() * 100)
            
        # imprimimos la pérdida de la época
        loss = np.mean(loss_hist)
        print(f'E{epoch:02} loss=[{loss:6.2f}]')

        
def train_model(build_model, epochs=5):
    set_seed()
    dl = build_dl()
    model = build_model()
    train(model, dl, epochs)

torch.Size([50, 2])
torch.Size([50, 1])
x shape=torch.Size([16, 2]) dtype=torch.float32
y shape=torch.Size([16, 1]) dtype=torch.float32


In [18]:
def build_high():
    model = nn.Linear(2,1)
    return model

build_high()

train_model(build_high, epochs=10)


E00 loss=[478.56]
E01 loss=[ 21.81]
E02 loss=[ 25.54]
E03 loss=[ 16.77]
E04 loss=[ 17.01]
E05 loss=[ 16.68]
E06 loss=[ 16.54]
E07 loss=[ 22.67]
E08 loss=[ 17.94]
E09 loss=[ 25.40]


In [20]:
tetas = list(build_high().parameters())[0][0]
sesgo = list(build_high().parameters())[1]


Para probar diferentes parámetros se usará la aquitectura de alto nivel

In [21]:
from itertools import product
batch_size_list = [4, 8, 16, 32, 64, 128]
shuffle_list = [True, False]
epochs_list = [5, 10, 15, 20]


In [22]:
resultados = pd.DataFrame({
})
for batch_size, epochs, shuffle in product(batch_size_list, epochs_list, shuffle_list):
    build_high()
    dl = build_dl(batch_size=batch_size, shuffle=shuffle)
    train_model(build_high, epochs=epochs)
    modelo = build_high()
    resultados = resultados.append(pd.DataFrame({
        "batch_size": [batch_size],
        "shuffle": [shuffle],
        "epochs": [epochs],
        "score": [F.mse_loss(modelo(x_trn),y_trn).item()],
        "modelo": [modelo],
        "parametros": [list(modelo.named_parameters())]
    }))

E00 loss=[478.56]
E01 loss=[ 21.81]
E02 loss=[ 25.54]
E03 loss=[ 16.77]
E04 loss=[ 17.01]
E00 loss=[478.56]
E01 loss=[ 21.81]
E02 loss=[ 25.54]
E03 loss=[ 16.77]
E04 loss=[ 17.01]
E00 loss=[478.56]
E01 loss=[ 21.81]
E02 loss=[ 25.54]
E03 loss=[ 16.77]
E04 loss=[ 17.01]
E05 loss=[ 16.68]
E06 loss=[ 16.54]
E07 loss=[ 22.67]
E08 loss=[ 17.94]
E09 loss=[ 25.40]
E00 loss=[478.56]
E01 loss=[ 21.81]
E02 loss=[ 25.54]
E03 loss=[ 16.77]
E04 loss=[ 17.01]
E05 loss=[ 16.68]
E06 loss=[ 16.54]
E07 loss=[ 22.67]
E08 loss=[ 17.94]
E09 loss=[ 25.40]
E00 loss=[478.56]
E01 loss=[ 21.81]
E02 loss=[ 25.54]
E03 loss=[ 16.77]
E04 loss=[ 17.01]
E05 loss=[ 16.68]
E06 loss=[ 16.54]
E07 loss=[ 22.67]
E08 loss=[ 17.94]
E09 loss=[ 25.40]
E10 loss=[ 23.52]
E11 loss=[ 19.21]
E12 loss=[ 32.48]
E13 loss=[ 16.36]
E14 loss=[ 18.31]
E00 loss=[478.56]
E01 loss=[ 21.81]
E02 loss=[ 25.54]
E03 loss=[ 16.77]
E04 loss=[ 17.01]
E05 loss=[ 16.68]
E06 loss=[ 16.54]
E07 loss=[ 22.67]
E08 loss=[ 17.94]
E09 loss=[ 25.40]
E10 loss=[

In [23]:
best_model = resultados.sort_values("score").reset_index().loc[0,"modelo"]


In [24]:
best_model(torch.tensor([3., 12]))


tensor([5.2098], grad_fn=<AddBackward0>)